### Step 0: Install and import libraries

In [1]:
# !pip install PyPDF2               # allows to use a template
# !pip install reportlab            # allows to add text to the template

In [2]:
from PyPDF2 import PdfWriter, PdfReader

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, landscape
import reportlab.rl_config
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

import io
import pandas as pd

### Step 1: Import data
Contains variables such as *name*, *course*, and *date* to be written to the certificate

In [3]:
participants = pd.read_excel('Input/participants.xlsx')
participants

,Student,Course,Date
0,Benito Martínez,Python,03 - 01 - 2023
1,Claire Cottrill,Power BI,06 - 01 - 2023
2,Emmanuelle Proulx,Tableau,09 - 01 - 2023
3,Greta Morgan,Excel,12 - 01 - 2023
4,Kevin Parker,R,15 - 01 - 2023
5,María Zardoya,SQL,18 - 01 - 2023


### Step 2, 3, 4, 5, 6, 7: Add text to certificate template

In [4]:
for i in range(len(participants)):
    
    ### Step 2: Create variables to be added to the certificate template
    student = participants.loc[i,'Student']
    course = participants.loc[i,'Course']
    date = participants.loc[i,'Date']
    
    ### Step 3: Set canvas conditions to add text to the template
    packet = io.BytesIO()
    width, height = letter # letter dimension is 612 x 792, so we will double it to write anywhere in the template
    c = canvas.Canvas(packet, pagesize=(width*2, height*2)) # Set dimensions where you can add text to PDF template
                                                            # double dimension to write anywhere in the template
    
    ### Step 4: Get text font types, more info at https://docs.reportlab.com/reportlab/userguide/ch3_fonts/#truetype-font-support
    reportlab.rl_config.warnOnMissingFontGlyphs = 0
    pdfmetrics.registerFont(TTFont('VeraBd', 'VeraBd.ttf')) # for variable 1: Student
    pdfmetrics.registerFont(TTFont('Vera', 'Vera.ttf'))     # for variable 2: Course
    pdfmetrics.registerFont(TTFont('VeraBI', 'VeraBI.ttf')) # for variable 3: Date
    
    ### Step 5: Set font features for each variable
    # Set font features for variable 1: Student
    c.setFillColorRGB(139/255,119/255,40/255) # Set font colour
    c.setFont('VeraBd', 50)                   # Set font type and font size
    c.drawCentredString(422, 310, student)    # Set position to write text ("watermark")
    
    # Set font features for variable 2: Course
    c.setFillColorRGB(139/255,119/255,40/255) # Set font colour
    c.setFont('Vera', 25)                     # Set font type and font size
    c.drawCentredString(422, 210, course)     # Set position to write text ("watermark")
    
    # Set font features for variable 3: Date
    c.setFillColorRGB(1/255,1/255,1/255)      # Set font colour
    c.setFont('VeraBI', 16)                   # Set font type and font size
    c.drawCentredString(578, 77, date)        # Set position to write text ("watermark")
    
    # Save all Canvas settings
    c.save()

    ### Step 6: Get final PDF certifciate
    # Get PDF template
    existing_pdf = PdfReader(open("Input/certificate_template.pdf", "rb"))   # Read PDF template
    page = existing_pdf.pages[0]              # Get first (in this case, the only one) page of the template
    
    # Add "watermark" (new_pdf) to template (page) 
    packet.seek(0)                            # Move to the beginning of the StringIO buffer
    new_pdf = PdfReader(packet)               # Create a new PDF with Reportlab based on "watermark"
    page.merge_page(new_pdf.pages[0])         # Add the "watermark" (which is the new PDF) on the existing PDF

    ### Step 7: Export final PDF certificate (page)
    file_name = student.replace(" ","_")
    certificado = "Certificates/" + file_name + "_certificate.pdf"
    outputStream = open(certificado, "wb")
    output = PdfWriter()
    output.add_page(page)
    output.write(outputStream)
    outputStream.close()